#### Loading everything from the model.py file 

In [1]:
from model import * 
### every module is accessible here.

#### Loading the Model from the model file

In [2]:
model = MyModel()

if torch.cuda.is_available():
    device = torch.device('cuda')  # Default CUDA device
    num_gpus = torch.cuda.device_count()
    print(f'{num_gpus} GPUs available')

    model = nn.DataParallel(model)  # Use multiple GPUs
    model.to(device)
else:
    print('CUDA is not available.')

model.eval() 

weights_path = '/mnt/e/codes_works/super_resolution/weights/modelvcu_epoch_48_val_loss_0.05541.pth' 

# Load the weights into the model
model.load_state_dict(torch.load(weights_path))

1 GPUs available


<All keys matched successfully>

#### Transform function to ready the frame for the model 

In [3]:
def get_transform():
    return Compose([
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

trans = get_transform()

#### Function for tensor to pillow image conversion

In [4]:
def tensor_to_image(tensor):
    tensor = tensor.cuda()

    # Define the mean and std
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)

    # Unnormalize the tensor
    std = std.cuda()
    mean = mean.cuda()
    tensor = tensor * std + mean

    # Clamp the values in the tensor to the range [0, 1]
    tensor = torch.clamp(tensor, 0, 1)

    # Convert the tensor to a PIL Image and then convert it to a numpy array
    image = transforms.ToPILImage()(tensor).convert("RGB")
    
    return image

#### Breaking up the video into frames

In [5]:
# Read the video file
cap = cv2.VideoCapture('/mnt/e/codes_works/super_resolution/videos/test_input.mp4')

# Create a Pillow Image object to store each frame
im = None
frames = list()

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # If the frame is empty, break the loop
    if not ret:
        break

    # Convert the frame to Pillow format
    im = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    test = trans(im)
    test = test.unsqueeze(0)
    test = test.cuda()

    out = model(test)

    img = tensor_to_image(out[0])

    frames.append(img)

#### Reconstructing the Upscaled images into a video

In [11]:
# Get the width and height of each frame
widths = []
heights = []
for i, fr in enumerate(frames):
    widths.append(fr.size[0])
    heights.append(fr.size[1])

# Use the largest width and height as the resolution of the output video
width = max(widths)
height = max(heights)
fps = 30

# cap.release()

out = cv2.VideoWriter('/mnt/e/codes_works/super_resolution/videos/test_output.mp4', cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))

# Write the frames to the video file
for i, fram in enumerate(frames):
    # Convert the PIL Image to a numpy array
    fram = np.array(fram)
    # OpenCV uses BGR color format, so we need to convert from RGB to BGR
    fram = cv2.cvtColor(fram, cv2.COLOR_RGB2BGR)
    out.write(fram)

# Release the VideoWriter object
out.release()

OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
